In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
from sklearn.model_selection import train_test_split
import numpy as np

# Загрузка данных из pickle-файлов
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

# Нормализация данных
X = X / 255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # Создание модели
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            # Компиляция модели
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            # Разделение данных на обучающий, валидационный и тестовый наборы
            X_train, X_temp, y_train, y_temp = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            # Обучение модели с валидацией
            model.fit(X_train, y_train,
                      batch_size=32,
                      epochs=10,
                      validation_data=(X_val, y_val),
                      callbacks=[tensorboard])

            # Оценка точности модели на тестовом наборе данных
            test_loss, test_acc = model.evaluate(X_test, y_test)
            print('Test accuracy:', test_acc)

            # Сохранение модели
            model.save('64x3-CNN.keras')


3-conv-64-nodes-0-dense-1710567480
Epoch 1/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.5623 - loss: 0.6750 - val_accuracy: 0.7044 - val_loss: 0.5874
Epoch 2/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.7234 - loss: 0.5526 - val_accuracy: 0.7418 - val_loss: 0.5208
Epoch 3/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7745 - loss: 0.4809 - val_accuracy: 0.7686 - val_loss: 0.4826
Epoch 4/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.7944 - loss: 0.4466 - val_accuracy: 0.7774 - val_loss: 0.4798
Epoch 5/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8082 - loss: 0.4082 - val_accuracy: 0.7600 - val_loss: 0.4811
Epoch 6/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.8317 - loss: 0.3779 - val_accuracy: 0.8020 - val_loss: 0.4406
Epoch 7/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.8415 - loss: 0.3550 - val_accuracy: 0.8071 - val_loss: 0.4420
Epoch 8/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - ac